In [30]:
import findspark
findspark.init()
import sys
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql.functions import *
from pyspark.sql.window import Window

In [2]:
import os
sparkClassPath = os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.postgresql:postgresql:42.1.1 pyspark-shell'

In [3]:
spark = SparkSession \
    .builder \
    .config("spark.driver.extraClassPath", sparkClassPath) \
    .getOrCreate()

In [4]:
actor = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://localhost:5432/postgres") \
    .option("driver", "org.postgresql.Driver") \
    .option("dbtable", "actor") \
    .option("user", "postgres") \
    .option("password", "postgre") \
    .load()

actor.show()

+--------+----------+------------+-------------------+
|actor_id|first_name|   last_name|        last_update|
+--------+----------+------------+-------------------+
|       1|  PENELOPE|     GUINESS|2022-02-15 10:34:33|
|       2|      NICK|    WAHLBERG|2022-02-15 10:34:33|
|       3|        ED|       CHASE|2022-02-15 10:34:33|
|       4|  JENNIFER|       DAVIS|2022-02-15 10:34:33|
|       5|    JOHNNY|LOLLOBRIGIDA|2022-02-15 10:34:33|
|       6|     BETTE|   NICHOLSON|2022-02-15 10:34:33|
|       7|     GRACE|      MOSTEL|2022-02-15 10:34:33|
|       8|   MATTHEW|   JOHANSSON|2022-02-15 10:34:33|
|       9|       JOE|       SWANK|2022-02-15 10:34:33|
|      10| CHRISTIAN|       GABLE|2022-02-15 10:34:33|
|      11|      ZERO|        CAGE|2022-02-15 10:34:33|
|      12|      KARL|       BERRY|2022-02-15 10:34:33|
|      13|       UMA|        WOOD|2022-02-15 10:34:33|
|      14|    VIVIEN|      BERGEN|2022-02-15 10:34:33|
|      15|      CUBA|     OLIVIER|2022-02-15 10:34:33|
|      16|

In [5]:
film_category = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://localhost:5432/postgres") \
    .option("driver", "org.postgresql.Driver") \
    .option("dbtable", "film_category") \
    .option("user", "postgres") \
    .option("password", "postgre") \
    .load()

In [6]:
category = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://localhost:5432/postgres") \
    .option("driver", "org.postgresql.Driver") \
    .option("dbtable", "category") \
    .option("user", "postgres") \
    .option("password", "postgre") \
    .load()

In [7]:
rental = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://localhost:5432/postgres") \
    .option("driver", "org.postgresql.Driver") \
    .option("dbtable", "rental") \
    .option("user", "postgres") \
    .option("password", "postgre") \
    .load()


In [8]:
inventory = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://localhost:5432/postgres") \
    .option("driver", "org.postgresql.Driver") \
    .option("dbtable", "inventory") \
    .option("user", "postgres") \
    .option("password", "postgre") \
    .load()


In [9]:
film = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://localhost:5432/postgres") \
    .option("driver", "org.postgresql.Driver") \
    .option("dbtable", "film") \
    .option("user", "postgres") \
    .option("password", "postgre") \
    .load()


In [10]:
film_actor = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://localhost:5432/postgres") \
    .option("driver", "org.postgresql.Driver") \
    .option("dbtable", "film_actor") \
    .option("user", "postgres") \
    .option("password", "postgre") \
    .load()



In [11]:
payment = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://localhost:5432/postgres") \
    .option("driver", "org.postgresql.Driver") \
    .option("dbtable", "payment") \
    .option("user", "postgres") \
    .option("password", "postgre") \
    .load()


In [61]:
city = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://localhost:5432/postgres") \
    .option("driver", "org.postgresql.Driver") \
    .option("dbtable", "city") \
    .option("user", "postgres") \
    .option("password", "postgre") \
    .load()

In [13]:
address = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://localhost:5432/postgres") \
    .option("driver", "org.postgresql.Driver") \
    .option("dbtable", "address") \
    .option("user", "postgres") \
    .option("password", "postgre") \
    .load()


In [14]:
customer = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://localhost:5432/postgres") \
    .option("driver", "org.postgresql.Driver") \
    .option("dbtable", "customer") \
    .option("user", "postgres") \
    .option("password", "postgre") \
    .load()

In [15]:
#Вывести количество фильмов в каждой категории, отсортировать по убыванию.
film_category.join(category,film_category.category_id == category.category_id,"inner")\
.groupBy('name').agg(count(col("film_id"))).orderBy(col('count(film_id)'),ascending = False)\
.show(truncate=False)

+-----------+--------------+
|name       |count(film_id)|
+-----------+--------------+
|Sports     |74            |
|Foreign    |73            |
|Family     |69            |
|Documentary|68            |
|Animation  |66            |
|Action     |64            |
|New        |63            |
|Drama      |62            |
|Games      |61            |
|Sci-Fi     |61            |
|Children   |60            |
|Comedy     |58            |
|Travel     |57            |
|Classics   |57            |
|Horror     |56            |
|Music      |51            |
+-----------+--------------+



In [17]:
#Вывести 10 актеров, чьи фильмы большего всего арендовали, отсортировать по убыванию.
rental.join(inventory,rental.inventory_id == inventory.inventory_id,"inner")\
    .join(film, inventory.film_id == film.film_id)\
    .join(film_actor, film_actor.film_id == film.film_id)\
    .join(actor, film_actor.actor_id == actor.actor_id)\
    .groupBy("first_name", 'last_name')\
    .agg(count(col("rental_id")))\
    .select('count(rental_id)',"first_name","last_name")\
    .orderBy(col('count(rental_id)'),ascending = False)\
    .limit(10)\
    .show(truncate=False)

+----------------+----------+-----------+
|count(rental_id)|first_name|last_name  |
+----------------+----------+-----------+
|825             |SUSAN     |DAVIS      |
|753             |GINA      |DEGENERES  |
|678             |MATTHEW   |CARREY     |
|674             |MARY      |KEITEL     |
|654             |ANGELA    |WITHERSPOON|
|640             |WALTER    |TORN       |
|612             |HENRY     |BERRY      |
|611             |JAYNE     |NOLTE      |
|605             |VAL       |BOLGER     |
|604             |SANDRA    |KILMER     |
+----------------+----------+-----------+



In [18]:
#Вывести категорию фильмов, на которую потратили больше всего денег.
payment.join(rental, payment.rental_id == rental.rental_id)\
    .join(inventory,rental.inventory_id == inventory.inventory_id)\
    .join(film, inventory.film_id == film.film_id)\
    .join(film_category, film_category.film_id == film.film_id)\
    .join(category, film_category.category_id == category.category_id)\
    .groupBy("name")\
    .sum("amount")\
    .select('name')\
    .orderBy(col('sum(amount)'),ascending = False)\
    .limit(1)\
    .show(truncate=False)

+------+
|name  |
+------+
|Sports|
+------+



In [20]:
#Вывести названия фильмов, которых нет в inventory. Написать запрос без использования оператора IN.
film.join(inventory,film.film_id == inventory.film_id,"left")\
.filter(inventory.inventory_id.isNull())\
.select(col('title'))\
.orderBy(col('title'),ascending = True)\
.show(truncate=False)

+----------------------+
|title                 |
+----------------------+
|ALICE FANTASIA        |
|APOLLO TEEN           |
|ARGONAUTS TOWN        |
|ARK RIDGEMONT         |
|ARSENIC INDEPENDENCE  |
|BOONDOCK BALLROOM     |
|BUTCH PANTHER         |
|CATCH AMISTAD         |
|CHINATOWN GLADIATOR   |
|CHOCOLATE DUCK        |
|COMMANDMENTS EXPRESS  |
|CROSSING DIVORCE      |
|CROWDS TELEMARK       |
|CRYSTAL BREAKING      |
|DAZED PUNK            |
|DELIVERANCE MULHOLLAND|
|FIREHOUSE VIETNAM     |
|FLOATS GARDEN         |
|FRANKENSTEIN STRANGER |
|GLADIATOR WESTWARD    |
+----------------------+
only showing top 20 rows



In [21]:
#Вывести топ 3 актеров, которые больше всего появлялись в фильмах в категории “Children”. 
#Если у нескольких актеров одинаковое кол-во фильмов, вывести всех.

t1 = actor.join(film_actor, actor.actor_id == film_actor.actor_id)\
.join(film_category, film_actor.film_id == film_category.film_id)\
.join(category, film_category.category_id == category.category_id)\
.filter(category.name == 'Children')\
.groupBy("first_name", 'last_name')\
.agg(count(col("last_name")))\
.select('last_name', 'first_name', 'count(last_name)')\
.orderBy('count(last_name)', ascending = False)

windowSpec = Window.orderBy('count(last_name)')

t2 = t1.withColumn("place",dense_rank().over(Window.orderBy(desc('count(last_name)'))))
t2.filter(col("place") < 4)\
.select('last_name', 'first_name', 'place').show()

+---------+----------+-----+
|last_name|first_name|place|
+---------+----------+-----+
|   VOIGHT|     HELEN|    1|
|    DAVIS|     SUSAN|    2|
|  GARLAND|     KEVIN|    3|
|     CRUZ|     RALPH|    3|
|     HURT|    WHOOPI|    3|
|    TANDY|      MARY|    3|
+---------+----------+-----+



In [22]:
#Вывести города с количеством активных и неактивных клиентов (активный — customer.active = 1). 
#Отсортировать по количеству неактивных клиентов по убыванию.
customers = city.join(address, address.city_id == city.city_id)\
.join(customer, customer.address_id == address.address_id)\
.withColumn('active',expr("CASE WHEN active = 1 THEN 1 ELSE 0 END"))\
.withColumn('inactive',expr("CASE WHEN active = 0 THEN 1 ELSE 0 END"))\
.select(col("*"))

In [23]:
active_customers = customers.groupBy('city')\
.sum('active')\
.select(col("*"))

In [24]:
inactive_customers = customers.groupBy('city')\
.sum('inactive')\
.select(col("*"))

In [25]:
active_customers.join(inactive_customers, inactive_customers.city == active_customers.city)\
.orderBy(col('sum(inactive)'),ascending = False).show()

+------------------+-----------+------------------+-------------+
|              city|sum(active)|              city|sum(inactive)|
+------------------+-----------+------------------+-------------+
|         Pingxiang|          0|         Pingxiang|            1|
|       Szkesfehrvr|          0|       Szkesfehrvr|            1|
|  Charlotte Amalie|          0|  Charlotte Amalie|            1|
|         Najafabad|          0|         Najafabad|            1|
|           Wroclaw|          0|           Wroclaw|            1|
|            Ktahya|          0|            Ktahya|            1|
|           Bat Yam|          0|           Bat Yam|            1|
|   Southend-on-Sea|          0|   Southend-on-Sea|            1|
|            Amroha|          0|            Amroha|            1|
|            Kamyin|          0|            Kamyin|            1|
|          Xiangfan|          0|          Xiangfan|            1|
|            Daxian|          0|            Daxian|            1|
|         

In [39]:
#Вывести категорию фильмов, у которой самое большое кол-во часов суммарной аренды в городах 
#(customer.address_id в этом city), и которые начинаются на букву “a”. То же самое сделать для городов в которых есть символ “-”.
rental_hours = rental.select(col("*"),
      datediff(col("return_date"),col("rental_date"))*24
    )

+---------+-------------------+------------+-----------+-------------------+--------+-------------------+-----------------------------------------+
|rental_id|        rental_date|inventory_id|customer_id|        return_date|staff_id|        last_update|(datediff(return_date, rental_date) * 24)|
+---------+-------------------+------------+-----------+-------------------+--------+-------------------+-----------------------------------------+
|        2|2022-05-24 23:54:33|        1525|        459|2022-05-28 20:40:33|       1|2022-02-16 03:30:53|                                       96|
|        3|2022-05-25 00:03:39|        1711|        408|2022-06-01 23:12:39|       1|2022-02-16 03:30:53|                                      168|
|        4|2022-05-25 00:04:41|        2452|        333|2022-06-03 02:43:41|       2|2022-02-16 03:30:53|                                      216|
|        5|2022-05-25 00:05:21|        2079|        222|2022-06-02 05:33:21|       1|2022-02-16 03:30:53|       

In [68]:
cities_a = category.join(film_category, film_category.category_id == category.category_id)\
.join(inventory, inventory.film_id == film_category.film_id)\
.join(rental_hours,inventory.inventory_id == rental_hours.inventory_id)\
.join(customer, customer.store_id == inventory.store_id)\
.join(address, customer.address_id == address.address_id)\
.join(city, address.city_id == city.city_id)\
.filter(upper(city.city).like("A%"))\
.groupBy(category.name)\
.sum('(datediff(return_date, rental_date) * 24)')

In [69]:
cities_a.select('name', "sum((datediff(return_date, rental_date) * 24))")\
.orderBy('sum((datediff(return_date, rental_date) * 24))', ascending = False)\
.limit(1)\
.show()

+------+----------------------------------------------+
|  name|sum((datediff(return_date, rental_date) * 24))|
+------+----------------------------------------------+
|Sports|                                       3170664|
+------+----------------------------------------------+



In [70]:
cities_hyphen = category.join(film_category, film_category.category_id == category.category_id)\
.join(inventory, inventory.film_id == film_category.film_id)\
.join(rental_hours,inventory.inventory_id == rental_hours.inventory_id)\
.join(customer, customer.store_id == inventory.store_id)\
.join(address, customer.address_id == address.address_id)\
.join(city, address.city_id == city.city_id)\
.filter(city.city.like("%-%"))\
.groupBy(category.name)\
.sum('(datediff(return_date, rental_date) * 24)')

In [71]:
cities_hyphen.select('name', "sum((datediff(return_date, rental_date) * 24))")\
.orderBy('sum((datediff(return_date, rental_date) * 24))', ascending = False)\
.limit(1)\
.show()

+------+----------------------------------------------+
|  name|sum((datediff(return_date, rental_date) * 24))|
+------+----------------------------------------------+
|Sports|                                       1786992|
+------+----------------------------------------------+

